In [1]:
! pip install scrapy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.5/264.5 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 2.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.6/93.6 kB 1.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 4.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.6-py3-none-any.whl size=11959 sha256=a969285c2f43ae38d31ef0dac475f7ed68c223524c1aba815384679f2b93c4be
  Stored in directory: /home/seun/.cache/pip/wheels/53/61/b0/7fcb8e55284c9c144d7c43619ecf2e75564424f9e9b0856e27
  Created wheel for zope.interfa

In [19]:
''' 
URL of the archive web-page which provides link to all solar-flares dates
in a text file. We crawl the webpage to extract all the 
needed information. 
 '''
url_1 = "https://www.ngdc.noaa.gov/stp/space-weather/solar-data/solar-features/solar-flares/x-rays/goes/xrs/goes-xrs-report_2013.txt"
url_2 = "https://www.ngdc.noaa.gov/stp/space-weather/solar-data/solar-features/solar-flares/x-rays/goes/xrs/goes-xrs-report_2014.txt"
url_3 = "https://www.ngdc.noaa.gov/stp/space-weather/solar-data/solar-features/solar-flares/x-rays/goes/xrs/goes-xrs-report_2016.txt"
url_4 = "https://www.ngdc.noaa.gov/stp/space-weather/solar-data/solar-features/solar-flares/x-rays/goes/xrs/goes-xrs-report_2017-ytd.txt"


# For web data scaping/data extraction and data restructuring
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy
import numpy as np
import re
import requests
import datetime

In [4]:
def extract_flare(url):
    response = requests.get(url)
    response.status_code
    flare_txt = response.text
    flare_list = re.findall('31777.{100}', flare_txt)
    return flare_list

In [21]:
def flare_data(flare_list):

              dfs  = []
              date = []
              mag  = []
              mag_val=[]

              for i in range(len(flare_list)):
                  global combined

                  # Scrap the data to extract the first 3 coloumns 
                  a=(flare_list[i])
                  s = [numpy.double(s) for s in re.findall(r'\d]+[.,\d]+|[\d]*[.][\d]+|[\d]+', a)]
                  s = s[0:3]
                  dfs.append(s)

                  #  Extract correct date format from the 1st coloumn and compile
                  da=(s[0])
                  da=da.astype(str)
                  da=da[5:]
                  da=pd.to_datetime(str(da)[-8:-2],format='%y%m%d')
                  date.append(da)
                  

                  # Extract the flare class and compile 
                  ma=list(a[50:61])
                  mag.append(ma)

                  # Extract the flare class magitude and compile
                  mag_va=(a[61:63])
                  mag_val.append(mag_va)

              # Convert the extracted data to DataFrame  
              df2 = pd.DataFrame(dfs)
              date2 = pd.DataFrame(date)
              ma = pd.DataFrame(mag)
              ma=ma.iloc[:,9]
              ma_va = pd.DataFrame(mag_val)
              

              # Merge the DataFRames together and assign names to each coloumn 
              result = pd.concat([date2, df2, ma, ma_va], axis=1)
              result.columns =['Flare_Date','Flare_date', 'Start_time', 'End_time','Flare_class', 'Flare_mag_value']
              result.drop(result.columns[[1]], axis=1, inplace=True)

             # Convert Flare_Date to year, month and day formart  
              result['year'] = result['Flare_Date'].dt.year
              result['month'] = result['Flare_Date'].dt.month_name()
              result['month_Number'] = result['Flare_Date'].dt.month

              result['day'] = result['Flare_Date'].dt.day
              return result


In [27]:

flare_list1=extract_flare(url_1)
flare_list2=extract_flare(url_2)
flare_list3=extract_flare(url_3)
flare_list4=extract_flare(url_4)

df1=flare_data(flare_list1)
df2=flare_data(flare_list2)
df3=flare_data(flare_list3)
df4=flare_data(flare_list4)


In [58]:
select = df1.loc[df1['Flare_class'] == 'B']
pd.set_option('display.max_rows', None)
select.head(1000)

,Flare_Date,Start_time,End_time,Flare_class,Flare_mag_value,year,month,month_Number,day
0,2013-01-01,58.0,111.0,B,78,2013,January,1,1
2,2013-01-01,1426.0,1456.0,B,52,2013,January,1,1
3,2013-01-01,1553.0,1607.0,B,62,2013,January,1,1
4,2013-01-01,2252.0,2301.0,B,47,2013,January,1,1
5,2013-01-02,921.0,934.0,B,56,2013,January,1,2
6,2013-01-02,2108.0,2111.0,B,42,2013,January,1,2
7,2013-01-03,408.0,429.0,B,57,2013,January,1,3
8,2013-01-03,1916.0,1938.0,B,79,2013,January,1,3
9,2013-01-03,2101.0,2117.0,B,94,2013,January,1,3
12,2013-01-04,206.0,217.0,B,82,2013,January,1,4


In [7]:
Im= "http://jsoc.stanford.edu/data/hmi/images"

import urllib.request

print('1- Create the url address to extract from  !!!!!!...')
uri = []

for k in range(len(df4)):
                  global combined
                  # Select from table 
                  yy=(df4['year'])
                  mn=(df4['month_Number'])
                  md=(df4['day'])
                  y= yy[k]
                  m= mn[k]
                  m= ('%02d' % m)
                  d= md[k]
                  d= ('%02d' % d)
                  y=y.astype(str)
                  #m=m.astype(str)
                  #d=d.astype(str)
                  ur= Im + "/" + y + "/"+ m + "/" + d
                  uri.append(ur)


1- Create the url address to extract from  !!!!!!...


In [8]:
uri[0]

'http://jsoc.stanford.edu/data/hmi/images/2017/01/04'

In [9]:
print('2- Begin image url extract !!!!!!...')                  
  
flare_ig=[]

for x in range(len(uri)):
     link= uri[x]

     url_image = link
     response = requests.get(url_image)
     response.status_code
     flare_txt = response.text
     flare_im = re.findall('href=.{33}', flare_txt)
     flare_im=flare_im[3:]
     flare_ig.append(flare_im)


2- Begin image url extract !!!!!!...
